<a href="https://colab.research.google.com/github/JuanAcevedo08/DeepLearning/blob/main/Clasificador_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clasificador de texto en pytorch


---

Pytorch ofrece soluciones a diferentes problemas como



*   torchaudio: Nos ofrece poder manipular audio y usar para entrenar modelos
*   torchtexto : Herramientas para manejar texto
*   torchvision : Herramientas para manejar imagenes
*   torchElastic : Cuando tenemos nuestros modelos deployeados estos nodos o maquinas que manejan cpus si una se cae o se agrega una puede caerse el modelo elastic se encarga de monitorear y controlarlas par aque no se caigan(entrenamiento distribuido y tolerante a fallos)
*   torchServe: ayuda a deployear modelos en produccion



# 1. Instalar librerias y alinearlas

In [ ]:
%%capture
!pip install torch==2.2.0 torchvision==0.17.2 torchaudio==2.2.0
!pip install torchtext==0.17.2 torchdata==0.7.1 portalocker>=2.0.0

In [ ]:
import torch, torchtext, torchdata

print("torch:", torch.__version__)
print("torchtext:", torchtext.__version__)
print("torchdata:", torchdata.__version__)

Explorar rapidmente los datos

In [ ]:
from torchtext.datasets import AG_NEWS #Importar dataset de texto a utilizar

In [ ]:
#Mostrar rapidamente el texto
train_ite = iter(AG_NEWS(split="train"))
next(train_ite)

# Procesar el texto



---
Tokenizacion
Vocab: id para cada token
vocab to id
embedding

In [ ]:
from torchtext.data.utils import get_tokenizer #Tokenizzador para el texto
from torchtext.vocab import build_vocab_from_iterator #Consturctor de vocabulario

tokenizer = get_tokenizer("basic_english") #Tipo de tokenizador
train_iter = AG_NEWS(split="train") #Instanciar datos de entrenamiento

In [ ]:
#Crear funcion que devuelve los toknes
def yield_token(data_iter):
  for _, text in data_iter:
    yield tokenizer(text) #Yield para retornar el texto tokenizado

#Crear vocabulario
vocab = build_vocab_from_iterator(yield_token(train_iter), specials=["<unk>"]) #Especal para cuando el modello tiene palabras que no sabe las marque como unknow
#Establecer que cada que no encuentre un token en el vocabulario lo lleve a unknow
vocab.set_default_index(vocab["<unk>"])

In [ ]:
#Probar elvocabulario con nuevas palabrass
tst_text = "Hello the day is kinda weird here in Colombia"
vocab(tokenizer(tst_text))

Funciones automatizadoras

In [ ]:
#Definir funciones para agilizar procesos
#Funcion lambda para darle un id del vocabulario si lo encuntra
text_to_id = lambda x: vocab(tokenizer(x))
#Revisar que funcione
print(text_to_id(tst_text))

#Funcion para labels, como python empiezza desde 0 si yo pongo un label 10 que es un texto este me devuelve el texto 11 no el 10 entonces 10 tendría que ser 9 para qeu me devuelva el 10
label_index = lambda n: int(n) - 1
print(label_index(10)) # -> tiene que ser 9

Funcion collate>fn

In [ ]:
#Definir funcion collatefn
#Empezar fijando el device, importante para operacione de tensores
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )

#Definir la función
def collate_batch(batch):
  """
  El modelo divide la totalidad de los datos en batches , a lo que voy a
  almacenar los datos en un solo tensor no en n batches diferentes como tensor para mi modelo

  labels_batch: Todas las etiquetas de la cantidad de batches que tengamos en ese momento ejemplo [1], [2], [3],[4]  // [1,2,3,4]
  tokens_batchs: Almacenará todos los tokens diferentes que ingresen en ese batch
  offsets: Indica donde va inicar la palabra ya que vamos a tener un vector comoe esto [palabra, algo ......] y ahí en uno estpan todos entonces tengo que indicar donde empiza cada frase diferente

  return: 3 Tensores torch en divice cuda o cpu para red neuronal
  """
  labels_batch = []
  tokens_batch = []
  offsets = [0] #Donde va empezar la palabra

  for (_label, _text) in batch:
    labels_batch.append(label_index(_label)) #Para que se ajuste el numero con el dato real
    process_text = torch.tensor(text_to_id(_text), dtype=torch.int64)
    tokens_batch.append(process_text)
    offsets.append(process_text.size(0))

  labels_batch = torch.tensor(labels_batch, dtype=torch.int64)
  offsets = torch.tensor(offsets[:-1], dtype=torch.int64).cumsum(dim=0) # Aplicar cmsum a solo filas para que indique real donde se encuentra el incio de la otra palraba ejemplo palabra 2 inicia en 5 palabra 3 inciia en 10 palabra 4 inicia en 0+5+10
  tokens_batch = torch.cat(tokens_batch) #Ya que tenemos en una lista muchso tensores entonces lo juntamos a solo uno [todos]

  return labels_batch.to(device) , tokens_batch.to(device), offsets.to(device)

In [ ]:
#EJEMPLO DE USO

# from torch.utils.data import DataLoader

# train_iter = AG_NEWS(split="train")
# dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch) #collatefn es para una funcion de recopilacion es decir ejemplo [1], [2], [3] = [1, 2, 3] los junta en un solo tensor

Creación del modelo

In [ ]:
#Importante las librerías
import torch.nn as nn
import torch.nn.functional as f  #Funciones creadas

#Modelo
class ModeltextClassifier(nn.Module):
  def __init__(self, vocab_size, embedding_dim,num_clases):
    super(ModeltextClassifier, self).__init__()

    #Primera capa procesador EMBEDDING
    self.embed = nn.EmbeddingBag(vocab_size, embedding_dim)
    #Capa de normalización  x - u / o para acelerar los procesos y tenga su escalas en el mismo rango y no sobreajuste
    self.bn = nn.BatchNorm1d(embedding_dim)
    #Capa de salida - Proyeccion lo lleva a una cantidad de clases especificas de salida
    self.fc = nn.Linear(embedding_dim, num_clases)

  def forward(self, tokens, offsets):
    #Realizar el embedding del texto
    embd = self.embed(tokens, offsets)
    #Aplicar normalzacion
    norm = self.bn(embd)
    #Aplicar funcion de activacion
    relu = f.relu(norm)
    #Definir la salida (Resultado , la classe que fue )
    return self.fc(relu)

En este punto el embed al ser un embedingbag relamente son 4 parametors que al inico e parecen pero son diferntes
perimero tenemos los parametros normales , de toda la vida , que se encargan de ajustar el tamaño y la cantidad de vectores a hacer

Ejemplo:
---



Parametros fijos:

Vocab_size=20 -> 20 Vectores a hacer

embeding_dim -> 10 -> Cada vector 10 espacios [1, 2, 3, 4, 5, 6, 7, 8, 9 ,10] con numeros random


---


Parametros  dinamicos:

Una vez puesto los otros ahora estos van a referirse a el batch es decir la divison a entrenar en ese momento

data -> Cantidad total de tokens dentro del batch la cantidad de palabras totales de las soraciones concatenadas

offsets -> indicará donde acaba cada frase para que el modelo pueda aplicar su embed



---


Ejemplo practico:

batch1:

datos = ['hola, amigo' , 'adios amigo']
tokens = ['hola', 'amigo'], ['adios']
vocab = {'hola':1, 'amigo':2, 'adios':3}

al pasarlos por el data loader y para que sea mas facil de procesar esto en la computadora todo termina siendo uno solo

[tensor con todos los datos numeros -vocab -id]

ya se sabe de que tamaño se hacen los embeds, pero no se sabe en donde hcer el embd entonces ya samos que hay 3 tokens pero no sabemos en donde aplicar el embed entonces entrn los offtense
token 1 a 2 hay una oracion la otra es otra entonces ahi ya sabemos de donde a donde se aplica el embd y se aplica

 [[embed1], [2] hasta completar todos los embeds  


 pero normalmente este hace los embeds por cada palabra no oracion entonces con embedgin bang este aplica na operacionsea max , mean , sum y crea solo uno

In [ ]:
clases = len(set([label for (label, text) in train_iter]))
model_t = ModeltextClassifier(vocab_size=len(vocab), embedding_dim=100, num_clases=clases).to(device)

In [ ]:
#Cantida de ids
id = len(vocab)
print(id)
#Arquitectura
print(model_t)
#Crear funcion para revisar la cantidad de parametros a ajustar
def params_count(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Cantidad de parametros a entrenar {params_count(model_t):,}")

Función de entrenamiento

In [ ]:
#Función entrenadora del modelo

def training(model, dataloader):
  #Configurar mood entrenamiento
  model.train()

  #Definir variables para controlar el modelo, ver precisón
  epoch_acc = 0
  epoch_loss = 0
  total_count = 0

  #Entrenar
  for index, (label, text, offsets) in enumerate(dataloader):
    #Restablecer el greadiente para recalcular nueva dirección de perdida
    optimizador.zero_grad()
    #Realizar predicciones
    prediccion_train = model(text, offsets)
    #Obtener perdida
    loss_train = criterio(prediccion_train, label)
    #Crear el backpropagation
    loss_train.backward()
    #Calcular cantidad de aciertos
    aciertos_train = (prediccion_train.argmax(1) == label).sum()#Elije la fila de indices con los valroes mas altos los quita de bool con .sum() convirtiendolos en si es true 1/0 de esa manera cada item y divide entre el dato real y da el accuracy
    #Evitar que los gradientes se eleven mucho
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) #Recorta los parametros , que llegeun hasta 01 para que no tenga valores muy altos
    #Realziar el re ajuste de los pesos
    optimizador.step()
    #realizar el accuracy
    epoch_acc += aciertos_train.item()
    epoch_loss += loss_train.item()
    total_count += label.size(0)

    #Revisar como vamos
    if index % 400 == 0 and index > 0:
        print(f'Epoch Numero: {epoch} | {index / len(dataloader)}% Batches completados | Perdida promedio: {epoch_loss / total_count} | Accuracy Promedio {epoch_acc / total_count}')

  return epoch_acc / total_count , epoch_loss / total_count

Funcion de evaluacion

In [ ]:
#Crear la funcion de evaluacion
def evaluation(modelo, dataloader):
  #Fijar metodo de evaluacion
  modelo.eval()
  #Variables de informacion
  epoch_acc_ev = 0
  epoch_loss_ev = 0
  total_labels_ev = 0

  #El no grad desactiva el calculo de gradiente lo cual es mas eficiente para realizar procesos de evlauacion liberando espacio en memoria
  with torch.no_grad():
    for index, (label, text, offsets) in enumerate(dataloader):
      #Realizar prediccion
      prediccion_test = modelo(text, offsets)
      #Calculo de variables informativas
      aciertos_test = (prediccion_test.argmax(1) == label).sum()
      #Calcular perdida del modelo
      loss_test = criterio(prediccion_test, label)
      #Actualizar variables locales
      epoch_acc_ev += aciertos_test.item()
      epoch_loss_ev += loss_test.item()
      total_labels_ev += label.size(0)

  return epoch_acc_ev / total_labels_ev, epoch_loss_ev / total_labels_ev

Crear los hyperparametros , Loss y optimizer y split

In [ ]:
#Hyperparams

EPOCHS = 15
LEARNING_RATE = 0.1
BATCH_SIZE = 64

In [ ]:
#Función de perdida
criterio = nn.CrossEntropyLoss()
#Optimizador
optimizador = torch.optim.SGD(params=model_t.parameters(), lr=LEARNING_RATE, momentum=0.9)

In [ ]:
#Separación en train, val, test
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset #Transformar de iterable a map para poder usar random split para que se pueda indexar y subdibidir | dandoles ñongitud e index

#Crear dataset train
train_iter, test_iter = AG_NEWS()
train_dt = to_map_style_dataset(train_iter)
test_dt = to_map_style_dataset(test_iter)

In [ ]:
#Datos para train set
train_size = int(len(train_dt) * 0.95) #95% datos para train
#Separación para train y test
split_train, split_eval = random_split(train_dt, [train_size, len(train_dt) - train_size]) #Fijamos de donde saca los datos y adema´s establecemos la cantidad de datos para train y eval
#Crear los dataloaders
from torch.utils.data import DataLoader
#Train dataloader
dataloader_train = DataLoader(split_train, BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
#Test dataloader
dataloader_test = DataLoader(test_dt, BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
#Val dataloader
dataloader_val = DataLoader(split_eval, BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

Entrenar el modelo

In [ ]:
#Crear variable que devuelve la perdida de validacióon mas pequeña
major_loss_validation = float('inf')

#Entrenar
for epoch in range(1, EPOCHS + 1 ):
  #Entrenar el modelo
  accuracy_mean_trian, mean_loss_train = training(model_t, dataloader_train)
  #Validación
  accuracy_mean_eval, mean_loss_eval = evaluation(model_t, dataloader_val)
  #Guardar el modelo con mejores resultados
  if mean_loss_eval < major_loss_validation:
    major_loss_validation = mean_loss_eval
    torch.save(model_t.state_dict(), "mejor_model.pt")

In [ ]:
acurracy_mean_test, loss_mean_test = evaluation(model_t, dataloader_test)

print(f'Promedio de accuracy para test: {acurracy_mean_test}')
print(f'Promedio de perdida para test: {loss_mean_test}')

Realizar inferencia

In [ ]:
# Mapeo de etiquetas para hacer inferencia con datos reales texto
labels_map = {
    1: "World",
    2: "Sports",
    3: "Business",
    4: "Sci/Tech"
}

#Crear funcón predictora

def predictora(text, convertor_id):
  with torch.inference_mode():
    text = torch.tensor(convertor_id(text), device=device)
    #Convencion pra modelo que usa compile para realizar esto mas rapido
    # optim_mod = torch.compile(model_t, mode='reduce_overhead') #Busca la forma mas rapida de ejecutar el modelo para que la compitlacion no sea tardada / Maxutotune para la compitlacion mas eficiente -> mas recursos
    salida = model_t(text, torch.tensor([0], dtype=torch.int64, device=device))#Al ser un texto el modelo va iniciar en el offset 0
    return salida.argmax(1).item() #Del tensor mas alto, el item mas alto

In [ ]:
# Función para probar diferentes ejemplos
labels_list = ['World', 'Sports', 'Business', 'Sci/Tech']
fk_db = [
    [
        "UN Security Council meets to discuss crisis in Middle East.",
        "European Union expands with new member states.",
        "Russia announces major military exercise near border.",
        "Thousands protest government reforms in Argentina.",
        "China signs trade agreement with African nations."
    ],
    [
        "Lakers defeat Celtics in overtime thriller.",
        "Roger Federer advances to Wimbledon final.",
        "Brazil beats Argentina in Copa America clash.",
        "Chicago Cubs win their first World Series in decades.",
        "Michael Phelps sets new world record in swimming."
    ],
    [
        "Amazon reports record quarterly profits.",
        "Wall Street closes higher after strong earnings reports.",
        "Toyota recalls vehicles due to safety concerns.",
        "Oil prices rise amid supply concerns in the Middle East.",
        "Microsoft announces acquisition of gaming company."
    ],
    [
        "NASA announces discovery of new exoplanet.",
        "Apple unveils latest iPhone model with new features.",
        "Google launches AI-powered translation tool.",
        "Scientists develop breakthrough cancer treatment.",
        "SpaceX successfully launches new rocket.",
        "Lionel Messi scored twice as Inter Miami defeated LA Galaxy 3-1 in the MLS, extending their unbeaten run to five games. The Argentine forward dazzled the crowd with a free-kick goal and an assist, proving once again why he remains one of the most influential players in world football."
    ]
]


def probar_ejemplos(db):
    for i, label_group in enumerate(db):
        print('-'*200)
        print(f'\nSección {labels_list[i]}')
        for text in label_group:
            predicted_class_index = predictora(text, text_to_id)
            predicted_class_name = labels_list[predicted_class_index]
            print(f'Text: {text}')
            print(f'Predicted class name: {predicted_class_name}\n')

#Ejecutando función creada
probar_ejemplos(fk_db)

Guardado y carga de modelos

el metodo state_dict() Se encarga en un diccionario de almacenar los parametros y loss sesgos del modelo para poder usarlo en otro modelo

In [ ]:
#Guardar modelo
model_stat_dict = model_t.state_dict()
#Guardar optimizador
optimizador_state_dict = optimizador.state_dict()
#Crear un checkpoint (Punto de guardado para luego entrenar)
checkpoint = {
    'model_state_dict ': model_stat_dict,
    'optimizador_state_dict': optimizador_state_dict,
    'epochs': EPOCHS,
    'loss': mean_loss_train
}

#Guardarlo
torch.save(checkpoint, 'model_saved_checkd.pth')

Exportar a huggin face

In [ ]:
%%capture
#No regrese todo lo que hace
!pip install hugginface_hub

In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# api.create_repo(repo_id="clasification-ag_news")

In [ ]:
#Subir la informacion del modelo
api.upload_file(path_or_fileobj='./model_saved_checkd.pth', path_in_repo='model_saved_checkd.pth', repo_id='JuanAcevedo/clasification-ag_news')